In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input
from keras.layers.embeddings import Embedding

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.initializers import RandomNormal

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
! unzip /content/drive/MyDrive/All/Participants_Data_DCW.zip

Archive:  /content/drive/MyDrive/All/Participants_Data_DCW.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [5]:
def clean_text(text):
    text = text.lower()
    
    
    text = re.sub(r'@[A-Za-z0-9_]+','', text)
    
    # Dealing with URL links
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex,'urlplaceholder', text)
    # A lot of url are write as follows: http bit.ly. Apply Regex for these cases
    utl_regex_2 = 'http [a-zA-Z]+\.[a-zA-Z]+'
    text = re.sub(utl_regex_2,'urlplaceholder', text)
    # Other formats: http : //t.co/ihW64e8Z
    utl_regex_3 = 'http \: //[a-zA-Z]\.(co|com|pt|ly)/[A-Za-z0-9_]+'
    text = re.sub(utl_regex_3,'urlplaceholder', text)
    
   
    
    
    # Contractions
    text = re.sub(r"what's", 'what is ', text)
    text = re.sub(r"can't", 'cannot', text)
    text = re.sub(r"\'s",' ', text)
    text = re.sub(r"\'ve", ' have ', text)
    text = re.sub(r"n't", ' not ', text)
    text = re.sub(r"im", 'i am ', text)
    text = re.sub(r"i'm", 'i am ', text)
    text = re.sub(r"\'re", ' are ', text)
    text = re.sub(r"\'d", ' would ', text)
    text = re.sub(r"\'ll", ' will ', text)
                  
    # Operations and special words  
    text = re.sub('#',' ', text)         
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub('foof', 'food', text)
    text = re.sub('msg', 'message', text)
    text = re.sub(' u ', 'you', text)
    
    # Ponctuation Removal
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    text = text.split()
    stop_words = stopwords.words("english")
    text = [tok for tok in text if tok not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(w) for w in text]
    return ' '.join(text)

In [6]:
# Cleaning Text
df['Review'] = df['Review'].map(lambda x: clean_text(x))

In [7]:
X = df['Review']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10,ngram_range=(1,5), max_features=300)
vectorizer.fit(X) # fit has to happen only on train data
X = vectorizer.transform(X)
X.shape
type(X)

scipy.sparse.csr.csr_matrix

In [9]:
X = X.toarray()
type(X)

numpy.ndarray

In [10]:
y = np.array(df[[ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']])
targets = [ 'Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability','Polarity']
y_label = df[targets].values

y_label.shape      
type(y)

numpy.ndarray

In [11]:
!pip install keras-tuner


     |████████████████████████████████| 98 kB 4.2 MB/s 


In [12]:
import kerastuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [13]:
# split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [14]:
def model_builder(hp):
  '''
  Args:
    hp - Keras tuner object
  '''
  batch_size = 256 # It is the sample size of inputs to be processed at each training stage. 
  hidden_units = 128
  dropout = 0.45
  X_in=X.shape[1]
  # Initialize the Sequential API and start stacking the layers
  model = Sequential()
  model.add(Dense(hidden_units, input_dim=X_in,activation='relu',
          kernel_initializer='he_uniform'))
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  model.add(Dropout(dropout))
  hp_units = hp.Int('units', min_value=280, max_value=290, step=1)
  
  model.add(Dense(units=hp_units,activation='relu', name='dense_002',
          kernel_initializer='he_uniform'))
  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(Dense(12))
  model.add(Activation('sigmoid'))
  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [15]:
# Instantiate the tuner
tuner = kt.Hyperband(model_builder, # the hypermodel
                     objective='val_accuracy', # objective to optimize
max_epochs=10,
factor=3, # factor which you have seen above 
directory='dir', # directory to save logs 
project_name='khyperband03')

In [16]:
# hypertuning settings
tuner.search_space_summary() 
# Output:- 

# Search space summary
# Default search space size: 2
# units (Int)
# {'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
# learning_rate (Choice)
# {'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 280, 'max_value': 290, 'step': 1, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [17]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# Perform hypertuning
tuner.search(X_train, y_train, epochs=2, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 04s]
val_accuracy: 0.004860267508774996

Best val_accuracy So Far: 0.5091130137443542
Total elapsed time: 00h 01m 18s
INFO:tensorflow:Oracle triggered exit


In [31]:
best_hp=tuner.get_best_hyperparameters()[0]

In [32]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hp)
h_model.summary()
h_model.fit(X_train, y_train, epochs=3, validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               38528     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_002 (Dense)           (None, 280)               36120     
                                                                 
 dropout_5 (Dropout)         (None, 280)               0         
                                                                 
 dense_5 (Dense)             (None, 12)                3372      
                                                                 
 activation_2 (Activation)   (None, 12)                0         
                                                                 
Total params: 78,020
Trainable params: 78,020
Non-trai

In [20]:
# h_model.save('tuner_model.h5')

In [33]:
h_eval_dict = h_model.evaluate(X_val, y_val, return_dict=True)

64/64 [==============================] - 0s 1ms/step - loss: 0.2279 - accuracy: 0.3610


In [34]:
yhat_test = h_model.predict(X_val)

model_log_loss=log_loss(y_val,yhat_test)
model_log_loss

3.7563976190745096

In [23]:
test_data = pd.read_csv('/content/test.csv')

In [24]:
pre_processed_test_review =  test_data['Review'].map(lambda x: clean_text(x))

In [25]:
test_data['preprocessed_test_review'] = pre_processed_test_review

In [26]:
test_vectorised_data = vectorizer.transform(test_data['preprocessed_test_review'] )
test_vectorised_data.shape

(2631, 300)

In [27]:
test_vectorised_data = test_vectorised_data.todense()

In [35]:
prediction_on_test_data = h_model.predict(test_vectorised_data)

In [36]:
df = pd.DataFrame(prediction_on_test_data, columns = ['Components', 'Delivery and Customer Support',
       'Design and Aesthetics', 'Dimensions', 'Features', 'Functionality',
       'Installation', 'Material', 'Price', 'Quality', 'Usability',
       'Polarity'])


In [37]:
df.to_csv('submission_tuner3.75639.csv', index=False)